In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 7.1 MB/s 
     |████████████████████████████████| 115 kB 73.5 MB/s 
     |████████████████████████████████| 101 kB 8.0 MB/s 
     |████████████████████████████████| 212 kB 35.2 MB/s 
     |████████████████████████████████| 127 kB 72.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [ ]:
import torch
from datasets import load_dataset, load_metric
squad_metric = load_metric('squad')
import math

In [ ]:
def preprocess(example):
    # This class preprocesses an example of the Squad dataset so that it works with word tokens
    
    # Data structure to be returned
    out = {}

    # The context and the question are tokenized by spaces.
    context_token = example['context'].strip().split(' ')
    question_token = example['question'].strip().split(' ')
    
    # We store in the data structure the tokens from the context and the questions
    out['context'] = context_token
    out['question'] = question_token
    
    if 'answers' not in example:
        return out
    
    # Now we search for the answers
    answer_start = example['answers']['answer_start']
    out['answers'] = []
    
    # And we calculate the start and end tokens of the answer
    for i, ans_st in enumerate(answer_start):
        c_token_len = len(example['context'][:ans_st].strip().split(' ')) # First we calculate the number of tokens until the answer start # strip removes leading/treading empty chars
        a_token_len = len(example['answers']['text'][i].strip().split(' ')) # Then we calculate the number of tokens in the answer
        out['answers'].append({'start' : c_token_len, 'end' : c_token_len + a_token_len})
    
    return out

# tokenizing conext with space and symbolic letter.
def tokenizing1(text):
    text = text.lower()
    tokens = []
    tokens_start = []
    tokens_len = []
    where_space = []
    i, token_start, token_len = 0, 0, 0
    
    while i < len(text):
        # is character alphbet?
        if text[i].isalpha(): # alphabet
            # Searching for the token length
            while True:
                # We search fot the end of the token to end the loop
                token_len += 1
                if i + token_len >= len(text) or not text[i + token_len].isalpha():
                    break
            # We keep track of the token, its lenght and the start
            tokens.append(text[i:i+token_len])
            tokens_len.append(token_len)
            tokens_start.append(i)
            i += token_len
            token_len = 0
        elif text[i] == ' ': # space
            where_space.append(i)
            i += 1
        else: # symbolic char
            # with a symbolic char we simply add it to the tokens being length 1
            tokens.append(text[i])
            tokens_len.append(1)
            tokens_start.append(i)
            i += 1
    # Returns the word tokens, where they start, their length and where there are spaces
    return tokens, tokens_start, tokens_len, where_space

def advanced_preprocess1(data):
    # calculates data structure with tokens their starts, lengths and the answer start and end
    
    out = {}
    # we tokenize context and answer
    tokens, tokens_start, tokens_len, where_space = tokenizing1(data['context'])
    tokens_q, _, _, _ = tokenizing1(data['question'])
    
    answer_start = data['answers']['answer_start']
    
    out = {'context' : tokens, 'question' : tokens_q, 'tokens_start' : tokens_start, 'tokens_len' : tokens_len, 'where_space' : where_space}
    
    out['answers'] = []
    
    for i, ans in enumerate(answer_start):
        start_index = tokens_start.index(ans)
        tokens, _, _, _ = tokenizing1(data['answers']['text'][i])
        end_index = start_index + len(tokens)
        out['answers'].append({'start' : start_index, 'end' : end_index})
    return out

def token2string(context, tokens_start, tokens_len, tokens_answers, where_space, test = []):
    # Converts list of tokens into string
    
    cont = ''

    for i, token in enumerate(context):
        # calculates forst the context
        if tokens_start[i] + tokens_len[i] in where_space:
            cont += (token + ' ')
        else:
            cont += token
    
    # calculating answer
    if len(test) > 0 :
            start_index = tokens_start[test[0]] if test[0] < len(context) else tokens_start[-1]
            end_index = tokens_start[test[1]] if test[1] < len(context) else tokens_start[-1]
    else:
        try:
            tokens_answers_start = tokens_answers['start']
            tokens_answers_end = tokens_answers['end']
            start_index = tokens_start[tokens_answers_start]
            end_index = tokens_start[tokens_answers_end-1] + tokens_len[tokens_answers_end -1]
        except:
             pass
    
    ans = cont[start_index : end_index]
    return cont, ans


In [ ]:

def preprocessing(squad_dataset):
	train_data = []
	del_train_data_index = []
	count = 0
    #preprocessing every example in the squad datset
	for i, data in enumerate(squad_dataset['train']):
		try:
			train_data.append(advanced_preprocess1(data))
		except:
			train_data.append([])
			del_train_data_index.append(i)
			count+=1
	        

	valid_data = []
	del_valid_data_index = []
	count = 0
	for i, data in enumerate(squad_dataset['validation']):
		try:
			valid_data.append(advanced_preprocess1(data))
		except:
			valid_data.append([])
			del_valid_data_index.append(i)
			count += 1

	return train_data, del_train_data_index, valid_data, del_valid_data_index
	


In [ ]:

import torch
import torch.nn as nn
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# word2id, id2word

def make_dict(train_data, del_train_data_index):

    # create a dictionary of word indexes
    word2id = {}
    tokens = []
    del_idx = []
    for i in range(len(train_data)):
        # skip if the example is invalid
        if i in del_train_data_index:
            continue
        # creating a list of tokens
        context_tokens = train_data[i]['context']
        question_tokens = train_data[i]['question']
        tokens.extend(context_tokens)
        tokens.extend(question_tokens)
        if not train_data[i]['answers']:
            del_idx.extend([i])
    # creating vocabulary
    vocab = ['UNK'] + list(set(tokens))
	    
    # dictionaries for conversion
    word2id = {word : id_ for id_ , word in enumerate(vocab)}
    id2word = {id_ : word for word, id_ in word2id.items()}

    return word2id, id2word

def wordToid(data):
    # converts strign tokens into indexes
    context = data['context']
    question = data['question']
    
    context = [word2id[word] if word in word2id else 0 for word in context]
    question = [word2id[word] if word in word2id else 0 for word in question]
    
    answer = []
    
    for dic in data['answers']:
        start = dic['start']
        end = dic['end']
        answer.append(context[start:end])
    return context, question, answer



def valid_answers(data_index, index = []):
    data = valid_data[data_index]
    context = data['context']
    tokens_start = data['tokens_start']
    tokens_len = data['tokens_len']
    where_space = data['where_space']
    
    if index[-1] > len(context):
        index[-1] = len(context)
    
    ans_tokens = context[index[0]:index[1]]

    ans = ''
    for i, token in enumerate(ans_tokens):
        if tokens_start[index[0] + i] + tokens_len[index[0] + i] in where_space:
            ans += (token + ' ')
        else:
            ans += token
    
    return ans.strip().lower()
    
def id2word_answer(data_index, start, end):
    ### valid 
    
    data = valid_data[data_index] 
    origin_data = squad_dataset['validation'][data_index]
    
    ans = valid_answers(data_index, [start,end])
    
    for i, a in enumerate(origin_data['answers']['text']):
        origin_data['answers']['text'][i] = a.lower()
    
    prediction_ = {'prediction_text': ans, 'id': origin_data['id']}
    reference = {'answers' : origin_data['answers'], 'id': origin_data['id']}
    
    return prediction_, reference


In [ ]:
# make batch . 

import numpy as np
import random
import torch
import torch.nn.functional as F

def make_batch(data, batch_size = 64, index = [], random = True, question = True):
    data = np.array(data)
    if random:
        indice = np.random.choice(len(data), batch_size, replace = False)
        for i, idx in enumerate(indice):
            if idx in del_train_data_index:
                indice[i] = 0
        data_batch = data[indice]
        
    else:
        for i,idx in enumerate(index):
            if idx in del_valid_data_index:
                index[i] = 0
        data_batch = data[index]

    context_max_len = 0
    question_max_len = 0
    
    for i in range(batch_size):
        if question:
            context_max_len = max(context_max_len, len(data_batch[i]['context']) + len(data_batch[i]['question']))
        else:
            context_max_len = max(context_max_len, len(data_batch[i]['context']))
    
    context_batch = []
    answer_start_batch = []
    answer_end_batch = []
    context_mask = []
    mask_loc = []
    
    for i, d in enumerate(data_batch):
        if d == []:
            continue
            
        context, questions, answer = wordToid(d)
        
        if question:
            context = np.concatenate([context, questions])
        
        context_len = len(context)
        context_padding = np.zeros(context_max_len - len(context))
        context = np.concatenate([context, context_padding])
        context_batch.append(context)
        
        for answers in data_batch[i]['answers']:
            answer_start_batch.append(answers['start'])
            answer_end_batch.append(answers['end'])
        
        context_mask.append(np.concatenate([np.ones(context_len), np.zeros(len(context_padding))], axis = 0))
        mask_loc.append(context_len)

    # print(len(torch.LongTensor(context_batch)),len(torch.LongTensor(answer_start_batch)),len(torch.LongTensor(answer_end_batch)),len(torch.LongTensor(context_mask)),len(torch.LongTensor(mask_loc)))
        
    return torch.LongTensor(context_batch), torch.LongTensor(answer_start_batch), torch.LongTensor(answer_end_batch), torch.LongTensor(context_mask), torch.LongTensor(mask_loc)

In [ ]:
class Attention(nn.Module):
    def __init__(self, embed_dim, num_heads = 4, dropout = 0.2, bias = True):
        super(Attention, self).__init__()
        
        self.embed_dim = embed_dim
        self.num_heads = num_heads 
        self.dropout = dropout
        self.bias = bias
        self.head_dim = embed_dim // num_heads
        
        self.k_proj = nn.Linear(embed_dim, embed_dim, bias = bias)
        self.q_proj = nn.Linear(embed_dim, embed_dim, bias = bias)
        self.v_proj = nn.Linear(embed_dim, embed_dim, bias = bias)
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias = bias)
        self.softmax = nn.Softmax(dim = -1)
    
    def scaled_dot_product(self, q, k, v, mask = None):
        # batch * seq * seq
        attention_weight = torch.matmul(q, k.transpose(-1,-2)) / math.sqrt(self.embed_dim)
        
        if mask is not None:
            attention_weight = attention_weight.masked_fill(mask == 0, -1e9)
        
        attention_weight = self.softmax(attention_weight)
        
        # batch * seq * embed_dim
        attention_output = torch.matmul(attention_weight, v)
        attention_output = self.out_proj(attention_output)
        
        return attention_output, attention_weight
    
    def forward(self, hidden, mask):
        query = self.q_proj(hidden) # batch * seq_len * hidden_dim(num_head * head_dim)
        key = self.k_proj(hidden) # batch * seq_len * hidden_dim(num_head * head_dim)
        value = self.v_proj(hidden) # batch * seq_len * hidden_dim(num_head * head_dim)
        
        
        
        attn_output, attn_weight = self.scaled_dot_product(query, key, value, mask)
        
        return attn_output, attn_weight

class LSTM_attn2(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_layers, bidirec = True, dropout = 0):
        super(LSTM_attn2, self).__init__()
        
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.num_direction = 2 if bidirec else 1
        
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        self.lstm = nn.LSTM(input_size = embed_dim, hidden_size = hidden_dim,
                           num_layers = num_layers, batch_first = True, dropout = dropout,
                           bidirectional = bool(bidirec))
        
        self.lin1 = nn.Linear(hidden_dim * self.num_direction, hidden_dim)
        
        self.attention = Attention(hidden_dim)
        
        self.lstm2 = nn.LSTM(input_size = hidden_dim, hidden_size = hidden_dim,
                           num_layers = num_layers, batch_first = True, dropout = dropout,
                            bidirectional = bool(bidirec))
        
        self.relu = nn.ReLU()
        self.out_lin = nn.Linear(hidden_dim * self.num_direction, 2)
    
    def subsequent_mask(self, batch, size, loc):
        attn_shape = (batch, size, size)
#         subsequent_mask = np.triu(np.ones(attn_shape), k = 1).astype('uint8')
        subsequent_mask = np.zeros(attn_shape).astype('uint8')
    
        for i in range(batch):
            subsequent_mask[i,:,loc[i]:] = 1

        return torch.from_numpy(subsequent_mask) == 0

    
    def forward(self, x, mask, loc):
        embed = self.embedding(x)
        h, c = self.init_states(x.size(0), self.num_direction)
        
        output, (hidden, cell) = self.lstm(embed, (h,c))
        output1 = self.lin1(output)
                
        mask = mask.unsqueeze(-1)
        output1 = output1 * mask
        
        attention_mask = self.subsequent_mask(x.size(0), x.size(1), loc) # batch , seq, mask location
        attention_mask = attention_mask.to(device)
        
        attn_output, attn_weight = self.attention(output1, attention_mask)
        
        output_attn = attn_output
        
        output_, (hidden2, cell2) = self.lstm2(output_attn, (hidden, cell))
        output_ = self.out_lin(output_)

        start = output_[:,:,0]
        end = output_[:,:,1]
        
        return start, end
        
    def init_states(self, batch_size, num_direction):
        return torch.zeros(self.num_layers * num_direction, batch_size, self.hidden_dim).to(device), torch.zeros(self.num_layers * num_direction, batch_size, self.hidden_dim).to(device)
        

        

In [ ]:
from tqdm import tqdm

def train(data, model, criterion, optimizer, batch_size = 32, num_iter = 30000, question = False, attention = True):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    model.to(device)
    model.train()
    loss_ = 0.
    acc_, start_acc, end_acc, val_score = 0, 0, 0, 0
    
    for i in tqdm(range(num_iter)):
        context, answer_start, answer_end, mask, loc = make_batch(data, batch_size, random = True, question = question)
        context, answer_start, answer_end, mask, loc = context.to(device), answer_start.to(device), answer_end.to(device), mask.to(device), loc.to(device)
#         context, answer_start, answer_end = context.to(device), answer_start.to(device), answer_end.to(device), #mask.to(device), loc.to(device) ###
        
        if attention:
            start, end = model(context, mask, loc)
        else:
            start, end = model(context, mask)

        
        loss_start = criterion(start, answer_start)
        loss_end = criterion(end, answer_end-1)
        
        loss = loss_start/2  + loss_end/2
        
        model.zero_grad()
        loss.backward()
        optimizer.step()

        # loss
        loss_ += loss.detach().cpu()
        
        # acc
        start_acc_ = (start.argmax(dim = -1) == answer_start).detach()
        end_acc_ = (end.argmax(dim = -1) == (answer_end -1)).detach()
        start_acc += start_acc_.sum().item()*1./batch_size
        end_acc += end_acc_.sum().item()*1./batch_size
        acc_ += (start_acc_ & end_acc_).sum().item()*1./batch_size
                
        if i % 1000 == 999 :
            print(f'{i + 1 : d}th iters >> loss = {loss_/1000:.4f}, start_acc = {start_acc/1000 * 100 : .4f}, end_acc = {end_acc/1000 * 100 : .4f}, acc = {acc_/1000 * 100 : .4f}')
            loss_, start_acc, end_acc, acc_ = 0., 0, 0, 0
            with torch.no_grad():
                model.eval()
                cur_val_score = validation(valid_data, model = model, batch_size = 128, question = question, attention = attention)
#                 if attention:
#                     if cur_val_score < val_score:
#                         optimizer.factor *= 0.8
#                 val_score = cur_val_score
                model.train()

    
    return model

In [ ]:
def validation(data, model, batch_size = 32, question = True, attention = True):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    predictions = []
    references = []


    for i in range(len(data)//batch_size):
        c, a_s, a_e, m, l= make_batch(data, batch_size = batch_size, index = np.arange(i*batch_size, (i+1)*batch_size), random = False, question = question)
        c, a_s, a_e, m, l = c.to(device), a_s.to(device), a_e.to(device), m.to(device), l.to(device)
        
        if attention:
            start, end = model(c,m,l)
        else:
            start, end = model(c, m) # batch_size * seq_len
        
        start_index = start.argmax(dim = 1).detach()
        end_index = end.argmax(dim = 1).detach()
        end_index += 1
        
        for j in range(batch_size):
            if i * batch_size + j in del_valid_data_index:
                continue
            if start_index[j] >= end_index[j]:
                continue
            pred, refer = id2word_answer(i * batch_size + j, start_index[j], end_index[j])
            predictions.append(pred)
            references.append(refer)

    results = squad_metric.compute(predictions = predictions, references = references)
    print('validation score : ', results)
    return results['f1']

In [ ]:
squad_dataset = load_dataset('squad')
train_data, del_train_data_index, valid_data, del_valid_data_index = preprocessing(squad_dataset)
word2id, id2word = make_dict(train_data, del_train_data_index) 


  0%|          | 0/2 [00:00<?, ?it/s]

```
64 42 42 64 64

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})
{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130313
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11858
    })
})
{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé Knowles', 'context': 'Beyoncé Giselle Knowles-Carter (nacida el 4 de septiembre de 1981) es una cantante, compositora, productora y actriz estadounidense. Nacida y criada en Houston, Texas, actuó en varios concursos de canto y baile cuando era niña, y saltó a la fama a finales de 1990 como cantante del grupo femenino de R & B Destiny \'s Child. Dirigida por su padre, Mathew Knowles, el grupo se convirtió en uno de los grupos de chicas más vendidos de todos los tiempos. Su hiato vio el lanzamiento del álbum debut de Beyoncé, Dangerously in Love (2003), que la estableció como una artista en solitario en todo el mundo, ganó cinco premios Grammy y presentó el Billboard Hot 100 sencillos número uno "Crazy in Love" y ".', 'question': '¿Cuándo Beyonce comenzó a ser popular?', 'answers': {'text': ['a finales de 1990'], 'answer_start': [246]}}

64 42 42 64 64


```





In [ ]:
vocab_size = len(word2id)
embed_dim = 300
hidden_dim = 128
num_layers = 2
bidirec = True, 
dropout = 0.5
print(vocab_size)
model = LSTM_attn2(vocab_size=vocab_size,embed_dim=embed_dim,hidden_dim=hidden_dim,num_layers=num_layers,bidirec=bidirec,dropout=dropout)

79423


In [ ]:
criterion=nn.CrossEntropyLoss()
learning_rate=0.5
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)
train(train_data, model, criterion, optimizer, batch_size = 64, num_iter = 100000, question = True, attention = True)

  0%|          | 0/100000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':
  1%|          | 999/100000 [03:10<4:58:55,  5.52it/s]

 1000th iters >> loss = 5.3442, start_acc =  2.4156, end_acc =  0.9406, acc =  0.0953


  1%|          | 1001/100000 [03:32<126:15:54,  4.59s/it]

validation score :  {'exact_match': 0.16224470318763123, 'f1': 8.134036164487732}


  2%|▏         | 1999/100000 [06:37<4:56:47,  5.50it/s]

...

 26%|██▌       | 25999/100000 [1:27:35<3:46:51,  5.44it/s]

 26000th iters >> loss = 5.0210, start_acc =  3.6984, end_acc =  1.9375, acc =  0.9984


 26%|██▌       | 26001/100000 [1:27:54<83:39:13,  4.07s/it] 

validation score :  {'exact_match': 1.3179571663920921, 'f1': 6.930406918806188}


 27%|██▋       | 26694/100000 [1:30:02<3:47:23,  5.37it/s]

In [ ]:
from google.colab import drive
drive.mount('./montura')

Code to save things and predict

In [ ]:
def predict(data, model, batch_size = 1, question = True):
    device = 'cpu' if torch.cuda.is_available() else 'cpu'
    predictions = []
    references = []


    for i in range(len(data)//batch_size):
        c, a_s, a_e, m, l= make_batch(data, batch_size = batch_size, index = np.arange(i*batch_size, (i+1)*batch_size), random = False, question = question)        
        
        start, end = model(c,m,l)
  
        
        start_index = start.argmax(dim = 1).detach()
        end_index = end.argmax(dim = 1).detach()
        end_index += 1
        
        for j in range(batch_size):
            if i * batch_size + j in del_valid_data_index:
                continue
            if start_index[j] >= end_index[j]:
                continue
            pred, refer = id2word_answer(i * batch_size + j, start_index[j], end_index[j])
            predictions.append(pred)
            references.append(refer)
    return predictions

In [ ]:
 import pickle
 class Library():
    def assign(self,word2id,id2word):
        self.word2id = word2id
        self.id2word = id2word
    def save(self):
        """save class as self.name.txt"""
        file = open('./montura/MyDrive/TFG/lstm_data.txt','wb')
        file.write(pickle.dumps(self.__dict__))
        file.close()

    def load(self):
        """try load self.name.txt"""
        file = open('./montura/MyDrive/TFG/lstm_data.txt','rb')
        dataPickle = file.read()
        file.close()

        self.__dict__ = pickle.loads(dataPickle)

In [ ]:
library = Library()
library.assign(word2id,id2word)
library.save()

In [ ]:
torch.save(model.state_dict(), './montura/MyDrive/TFG/lstm_model')


In [ ]:
library = Library()
library.load()

In [ ]:
question = 'When the three types of rock are re-melted what is formed?'
context = 'There are three major types of rock: igneous, sedimentary, and metamorphic. The rock cycle is an important concept in geology which illustrates the relationships between these three types of rock, and magma. When a rock crystallizes from melt (magma and/or lava), it is an igneous rock. This rock can be weathered and eroded, and then redeposited and lithified into a sedimentary rock, or be turned into a metamorphic rock due to heat and pressure that change the mineral content of the rock which gives it a characteristic fabric. The sedimentary rock can then be subsequently turned into a metamorphic rock due to heat and pressure and is then weathered, eroded, deposited, and lithified, ultimately becoming a sedimentary rock. Sedimentary rock may also be re-eroded and redeposited, and metamorphic rock may also undergo additional metamorphism. All three types of rocks may be re-melted; when this happens, a new magma is formed, from which an igneous rock may once again crystallize.'

In [ ]:
data = {
    'train':{
        'id':[''],
        'title':[''],
        'answer':[''],
        'question':[''],
        'context':['']
    },
    'validation':{
        'id':[''],
        'title':[''],
        'answer':[''],
        'question':[question],
        'context':[context]
    }
}
train_data, del_train_data_index, valid_data, del_valid_data_index = preprocessing(data)
print(train_data, del_train_data_index, valid_data, del_valid_data_index)


In [ ]:
vocab_size = len(library.word2id)
embed_dim = 300
hidden_dim = 128
num_layers = 2
bidirec = True, 
dropout = 0.5
model = LSTM_attn2(vocab_size=vocab_size,embed_dim=embed_dim,hidden_dim=hidden_dim,num_layers=num_layers,bidirec=bidirec,dropout=dropout)
model.load_state_dict(torch.load('./montura/MyDrive/TFG/lstm_model'))
model.eval()

In [ ]:
results = predict(valid_data,model)
results